Import libraries

In [1]:
import xgboost as xgb
import pandas as pd
from sklearn.metrics import accuracy_score

import hopsworks

In [ ]:
project = hopsworks.login()

# Get the model registry
mr = project.get_model_registry()

In [ ]:
# Retrieve the model from the model registry
retrieved_model = mr.get_model(
    name="news_propagation_model",
    version=3,
)

# Download the saved model files to a local directory
saved_model_dir = retrieved_model.download()

In [ ]:
# Initialize the model
model = xgb.XGBClassifier()

# Load the model from a saved JSON file
model.load_model(saved_model_dir + "/model.json")
model

In [ ]:
feature_view = retrieved_model.get_feature_view()